# COMP2002 Report

The coursework has two parts - one is a machine learning exercise and the second is about evolutionary computation.

- [ ] Part 1: Machine Learning
    - [ ] 1.1 Data preparation (10%)
    - [ ] 1.2 Classification (20%)
    - [ ] 1.3 Assessment of classification (30%)
- [ ] Part 2: Optimisation
    - [ ] 2.1 Generation of random solutions (10%)
    - [ ] 2.2 Algorithm implementation (25%)
    - [ ] 2.3 Visualisation of results (15%)

## Part 1: Machine Learning

You have been provided with datasets relating to glass. Your task is to train and assess classification models that predict the type of glass based on 9 inputs.

- Data preparation
- Classification
- Assessment of classification

### Task 1.1 - Data preparation

The first phase of the work requires you to load the data you have been provided with into your selected program. Before the data can be used to train and test your models you must first prepare it – this means that the inputs must be converted to a suitable format. There is no missing data in the dataset.

> The data is loaded and normalised, and the code to do so is efficiently written.

In [ ]:
import numpy as np
import pandas as pd

### Task 1.2 – Classification

Having prepared the data you must now build a classification tool that can predict new points. Use the following classification implementations within the respective program packages to construct predictors for the dataset:
- kNN
- SVM

For each of the models, you must investigate the optimal number of neighbours and an appropriate kernel shape from linear, polynomial and radial. You must demonstrate that each classifier can provide a prediction for a given input.

Note: for the polynomial kernel, restrict your investigation to polynomials of degree 2 and 3. 

> Both models are trained and tested correctly. The code used to do so is well organised and efficient.

### Task 1.3 – Assessment of classification 

After identifying the best model parameters in the previous task, the classification models you have implemented must be assessed. To do this you are required to assess the accuracy for each model. You may use the accuracy implementation available to do this. It is not sufficient to report a single accuracy score. You must use cross-validation to report training results and report these values using a plot. You will also need to write a summary analysing your results and findings. 

> Accuracy is used in combination with crossvalidation, and the results are presented with a boxplot. The code is efficient and well organised. Analysis of the results is complete and well understood. 

## Part 2 – Optimisation

The second part of this assignment requires you to construct an algorithm that can optimise single-objective optimisation problems. 

The problems are as follows:

Problem | Formulation
--- | ---
Schaffer Function N.2 | $f(x, y) = 0.5 + \frac{\sin(x^2-y^2)^2 - 0.5}{[1+0.001(x^2+y^2)]^2}$
Modified Ackley Function | $f(x, y) = -20 \exp \left(-0.2 \sqrt{0.5 (x^2 + y^2)} \right) - \exp \left[ 0.5 \left(\cos(2\pi x) + \cos(2\pi y) \right) \right] + 20$
Easom Function | $f(x, y) = -\cos(x) \cos(y) \exp \left(-\left( (x - \pi)^2 + (y - \pi)^2 \right) \right)$

Each solution should have $D = 2$ continuous decision variables that can take any value (you are recommended to start with random values between -5 and 5 for all three problems).


### Task 2.1 – Generation of random solutions

You must use an evolutionary algorithm (EA). When evaluating an EA it is standard to compare against randomness. Generate 500 random solutions to the problem and plot their fitness values. You should plot the x and y values and colour the solutions according to their fitness value.

> The data is loaded and the fitness function is correct. Random solutions are generated correctly. The code to do so is efficiently structured.

### Task 2.2 – Algorithm implementation

You should implement a population-based evolutionary algorithm as described in the lectures. Your algorithm must have the following features:
- A crossover operator that performs uniform crossover.
- A mutation operator that performs an additive Gaussian mutation.
- A selection operator that combines a generation’s parent and child populations and identifies the parent solutions for the next generation.

> The mutation operators and algorithm have been implemented correctly. The fitness archive stores fitnesses correctly. The code is efficiently structured and well organised.

### Task 2.3 – Visualisation of results

Modify your optimiser to record the average fitness at each generation. Then, after your optimiser has run, produce a plot showing the change in average fitness over the runtime of the algorithm.

Your visualization code must be separate from the optimiser.

> A graph shows the correct results and the correct experimental setup has been followed.

***

## Resources

[Genertic Algorithms Explained By Example](https://www.youtube.com/watch?v=uQj5UNhCPuo)
<br>

[Grokking Algorithms](C:/Users/richa/Desktop/ebooks/Aditya%20Bhargava%20-%20Grokking%20Algorithms_%20An%20illustrated%20guide%20for%20programmers%20and%20other%20curious%20people-Manning%20Publications%20(2016).pdf)